# Menampilkan Waktu & Posisi Hilal dari Suatu Tempat

In [1]:
# @_jengkolrebus
# Curup, Bengkulu
# Mei 2020

In [2]:
from skyfield.api import load, Topos
from skyfield import almanac
from datetime import datetime, timedelta
from pytz import timezone
from ipywidgets import interact
from ipywidgets import widgets as wg
import calendar
import pandas as pd
from IPython.display import display, HTML

In [3]:
conj = []
sunset = []
bulan_alt = []
bulan_az = []
matahari_alt = []
matahari_az = []


jkt = timezone('Asia/Jakarta')
ts = load.timescale()
e = load('de421.bsp')    # ephemeris

bulan = e['moon']
bumi = e['earth']
lat='7.83305556 S'
long='110.38305556 E'

obsUAD = Topos(lat, long)
loc = bumi + obsUAD

In [8]:
class var:
    bulan = e['moon']
    bumi = e['earth']
    def __init__ (self, loc):
        self.loc = loc
        
loc = obsUAD + var.bumi

In [9]:
def getTopos(Latitude, Longitude):
    global obsUAD
    lat=Latitude
    long=Longitude
    obsUAD = Topos(lat, long)
    loc = obsUAD + var.bumi
    a = var(loc)
    interact(konjungsi, t0=dari, t1=sampai)

# fungsi mendapatkan waktu konjungsi

def konjungsi(t0, t1):
    conj.clear()
    sunset.clear()
    bulan_alt.clear()
    bulan_az.clear()
    matahari_alt.clear()
    matahari_az.clear()
    t0 = ts.utc(t0)
    t1 = ts.utc(t1)
    f = almanac.oppositions_conjunctions(e, bulan)
    t, y = almanac.find_discrete(t0, t1, f)

    for ti, yi in zip(t, y):
        if (yi==1):
            conj.append(ti.astimezone(jkt))
            t = ti.utc
            t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
            t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
            t_rs, y_rs = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(e, obsUAD))
            for t_sun, y_sun in zip(t_rs, y_rs):
                if (y_sun == False):
                    loc = a.loc
                    sunset.append(t_sun.astimezone(jkt))
                    t = t_sun.utc
    
                    astrometrik_bulan = loc.at(ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])).observe(var.bulan)
                    astrometrik_matahari = loc.at(ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])).observe(e['sun'])

                    alt_bulan, az_bulan, jarak_bulan = astrometrik_bulan.apparent().altaz()
                    alt_matahari, az_matahari, jarak_matahari = astrometrik_matahari.apparent().altaz()

                    bulan_alt.append(alt_bulan)
                    bulan_az.append(az_bulan)
                    matahari_alt.append(alt_matahari)
                    matahari_az.append(az_matahari)
                    
#                     tabel = list(zip(conj, sunset, bulan_alt, bulan_az, matahari_alt, matahari_az))
#                     df = pd.DataFrame(tabel, columns=['Waktu Konjungsi (UTC+07)', 'Waktu Hilal (UTC+07)', 'Altitude Bulan', 'Azimuth Bulan', 'Altitude Matahari', 'Azimuth Matahari'])
#                     df
                    
                else:
                    pass
            
            
        else:
            pass
        
    #data(conj, sunset, bulan_alt, bulan_az, matahari_alt, matahari_az)
    tabel = list(zip(conj, sunset, bulan_alt, bulan_az, matahari_alt, matahari_az))
    df = pd.DataFrame(tabel, columns=['Waktu Konjungsi (UTC+07)', 'Waktu Hilal (UTC+07)', 'Altitude Bulan', 'Azimuth Bulan', 'Altitude Matahari', 'Azimuth Matahari'])
    df.index+=1
    display(df)

In [10]:
# wg.interact(getLat, Latitude='7.83305556 S')
# wg.interact(getLong, Longitude='110.38305556 E')

hari_ini = datetime.now().date()
days_in_month = calendar.monthrange(hari_ini.year, hari_ini.month)[1]
besok = hari_ini + timedelta(days_in_month)

dari = wg.DatePicker(value=hari_ini,
    description='Dari:',
    disabled=False
)
sampai = wg.DatePicker(value=besok,
    description='Sampai:',
    disabled=False
)

interact(getTopos, Latitude='7.83305556 S', Longitude='110.38305556 E')
#interact(konjungsi, t0=dari, t1=sampai)

interactive(children=(Text(value='7.83305556 S', description='Latitude'), Text(value='110.38305556 E', descrip…

<function __main__.getTopos(Latitude, Longitude)>